<a href="https://colab.research.google.com/github/dead96pool/stress_detection_mlai/blob/main/MLAI_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# git repo clone
!git clone https://github.com/dead96pool/datasets
# removing the clone
#!rm -rf datasets

fatal: destination path 'datasets' already exists and is not an empty directory.


In [ ]:
calm_path = "/content/datasets/machine_learning/mlai_project/2019_stress_detection_dataset/CALM"
stress_path = "/content/datasets/machine_learning/mlai_project/2019_stress_detection_dataset/STRESS"